# <img src="./image/taxi_logo.png" alt="drawing" width="30"/>Taxi Driver Go Where?
### A Time Series Prediction on the Demand of Taxi at New York City
by Tan Jun Pin

## Datasets Sources
All datasets were obtained from NYC.GOV with Taxi & Limousine Commission.
([source](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page))

There are total of 2 types of datasets were downloaded which are:
* [`Yellow Taxi Trip Records (Jan' 22 to Aug' 22)`](./datasets/yellow_taxi_2022)
* [`Taxi Zoning and Map`](./datasets/taxi_zones)


### Project Background

Taxi services are very customer-centric. Everyone one of us could easily get a taxi ride these days. We could easily hail down any taxi on the streets or even with just few buttons away using applications. Taxi services provide us with much convenient of transporting us to any location (as long as the road connects!). As a passenger, once we reach our destination, the taxi will be empty and the driver is ready to 'hunt' for new passengers.

However, it is not easy to get a new passenger especially the taxi is not located at any popular location like business district, airport or shopping areas. Very often, taxi drivers are 'abandoned' at areas where they are not familiar or even far from the 'hotspot'. It is all known that a taxi has to keep on cruise on the road or at least driver towards the famous hotspot to get new passengers. 

Crusing on the road is time and fuel consuming which is not ideal for taxi drivers. Worst is when the driver is cruising on the wrong area where the area itself has no demand for taxi at that particular time. Demand for Taxi is not always constant throughout the day. Area like the business district could see more Pick Up in the evening as everyone is returning home while Airport on the other hand, could have a more constant pick up demand due to the passengers arriving to the airport. On the other hand, some areas like factory district or residential areas, the demand for taxi is simply not good throughout the day.

Hence, in this project, we would like to predict on the demand of Taxi focusing on the areas in New York City where taxi drivers could easily know the area they are located or going to go will have any demand for taxi. They could also know if the neighboring areas will have any demand for taxi.

# Data Import & Cleaning

## Importing library for Cleaning

In [1]:
from glob import glob
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

## Importing Datasets

In [2]:
# To import all the datasets in the folder
files = glob('./datasets/yellow_taxi_2022/*')
y_22 = {}
N = 1
for i in files:
    y_22[N] = pd.read_parquet(i)
    N += 1

In [3]:
# To join all the tables into a single DataFrame
yellow_22 = pd.DataFrame()
for n in range(1,N):
    yellow_22 = pd.concat([yellow_22, y_22[n]], ignore_index = True, sort = False)
    print(yellow_22.shape)

(3152677, 19)
(5616608, 19)
(9244490, 19)
(12223921, 19)
(15398315, 19)
(18956439, 19)
(22556359, 19)
(26144654, 19)


In [4]:
# To check on the datasets
yellow_22.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-08-01 00:17:39,2022-08-01 00:19:58,1.0,0.4,1.0,N,114,148,1,3.5,3.0,0.5,1.80,0.00,0.3,9.10,2.5,0.0
1,1,2022-08-01 00:26:06,2022-08-01 00:31:55,1.0,1.4,1.0,N,79,137,1,6.5,3.0,0.5,2.05,0.00,0.3,12.35,2.5,0.0
2,1,2022-08-01 00:45:49,2022-08-01 00:59:29,1.0,5.2,1.0,N,79,74,1,16.0,3.0,0.5,3.95,0.00,0.3,23.75,2.5,0.0
3,1,2022-08-01 00:05:49,2022-08-01 00:25:42,1.0,9.4,1.0,N,138,113,1,28.0,3.0,0.5,7.65,6.55,0.3,46.00,2.5,0.0
4,1,2022-08-01 00:36:29,2022-08-01 00:51:29,1.0,1.7,1.0,N,137,68,1,11.0,3.0,0.5,1.00,0.00,0.3,15.80,2.5,0.0


In [5]:
# To arrange the datasets based on the pickup_datetime
yellow_22.sort_values(by ='tpep_pickup_datetime', inplace = True, ignore_index = True)

In [6]:
# To check on the datasets
yellow_22.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2001-01-01 00:03:14,2001-01-01 01:12:47,1.0,20.77,2.0,N,132,230,2,52.0,0.0,0.5,0.0,6.55,0.3,61.85,2.5,0.0
1,2,2001-01-01 00:27:45,2001-01-01 00:34:17,1.0,0.88,1.0,N,48,143,2,6.0,1.0,0.5,0.0,0.00,0.3,10.30,2.5,0.0
2,2,2001-01-01 01:02:18,2001-01-01 17:20:19,1.0,3.10,1.0,N,237,234,1,14.0,1.0,0.5,0.0,0.00,0.3,18.30,2.5,0.0
3,2,2001-01-01 01:23:51,2001-01-01 01:50:08,1.0,4.68,1.0,N,230,231,2,20.5,1.0,0.5,0.0,0.00,0.3,24.80,2.5,0.0
4,2,2001-01-01 01:52:48,2001-01-01 17:37:26,1.0,6.95,1.0,N,231,264,1,22.0,1.0,0.5,3.5,0.00,0.3,29.80,2.5,0.0


## Data Infomation

To identify the number of rows & columns for each datasets and the features

In [7]:
def row_col(pd):
    print(pd.shape)
    return (pd.head())

In [8]:
row_col(yellow_22)

(26144654, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2001-01-01 00:03:14,2001-01-01 01:12:47,1.0,20.77,2.0,N,132,230,2,52.0,0.0,0.5,0.0,6.55,0.3,61.85,2.5,0.0
1,2,2001-01-01 00:27:45,2001-01-01 00:34:17,1.0,0.88,1.0,N,48,143,2,6.0,1.0,0.5,0.0,0.00,0.3,10.30,2.5,0.0
2,2,2001-01-01 01:02:18,2001-01-01 17:20:19,1.0,3.10,1.0,N,237,234,1,14.0,1.0,0.5,0.0,0.00,0.3,18.30,2.5,0.0
3,2,2001-01-01 01:23:51,2001-01-01 01:50:08,1.0,4.68,1.0,N,230,231,2,20.5,1.0,0.5,0.0,0.00,0.3,24.80,2.5,0.0
4,2,2001-01-01 01:52:48,2001-01-01 17:37:26,1.0,6.95,1.0,N,231,264,1,22.0,1.0,0.5,3.5,0.00,0.3,29.80,2.5,0.0


Due to large datasize, rows that is deemd to be not irrelevant to the datasets will be removed from the dataframe

The following features will be removed from the dataset:
- VendorID: Code indicating the TPEP Provider that provided the record
- Passenger_count: The number of passengers in the vehicle. 
- store_and_fwd_flag : This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward”
- payment_type: A numeric code signifying how the passenger paid for the trip. 
- RatecodeID: The final rate code in effect at the end of the trip
- mta_tax: \\$0.50 MTA State Surcharge for all trips that end in New York City or Nassau, Suffolk, Westchester, Rockland, Dutchess, Orange or Putnam Counties.
- toll_amount: Total amount of all tolls paid in trip. 
- improvement_surcharge: $0.30 improvement surcharge assessed trips at the flag drop.
- congestion_surcharge: Total amount collected for all trips that begin, end or pass through Manhattan south of 96th Street.
- airport_fee: As Newark Airport is outside of New York City, this information is not relevant to this project

In [9]:
yellow_22.drop(labels = ['VendorID', 'passenger_count', 'RatecodeID', 
                    'store_and_fwd_flag', 'payment_type', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 
                    'congestion_surcharge', 'airport_fee'], axis = 1, inplace = True)

In [10]:
row_col(yellow_22)

(26144654, 9)


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,extra,tip_amount,total_amount
0,2001-01-01 00:03:14,2001-01-01 01:12:47,20.77,132,230,52.0,0.0,0.0,61.85
1,2001-01-01 00:27:45,2001-01-01 00:34:17,0.88,48,143,6.0,1.0,0.0,10.30
2,2001-01-01 01:02:18,2001-01-01 17:20:19,3.10,237,234,14.0,1.0,0.0,18.30
3,2001-01-01 01:23:51,2001-01-01 01:50:08,4.68,230,231,20.5,1.0,0.0,24.80
4,2001-01-01 01:52:48,2001-01-01 17:37:26,6.95,231,264,22.0,1.0,3.5,29.80


## Reduce the Dataframe Size

In [11]:
# To check on the memory usage of the dataset
ori_size = yellow_22.memory_usage().sum()/(1024*1024)

Upon Removal of uncessary columns, the dataset is still having size of 1.9GB which is still large. 
Hence, the type of each feature will be engineered to datatype where size is smaller.

In [12]:
# To identify the datatype of each feature
yellow_22.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
trip_distance                   float64
PULocationID                      int64
DOLocationID                      int64
fare_amount                     float64
extra                           float64
tip_amount                      float64
total_amount                    float64
dtype: object

### Categorical Dataset

Object data type utilizes big memory space and in this dataset, the categorical features below will be then converted to `category` type:
- PULocationID: TLC Taxi Zone in which the taximeter was engaged
- DOLocationID : TLC Taxi Zone in which the taximeter was disengaged

In [13]:
for col in ['PULocationID', 'DOLocationID']:
    yellow_22[col] = yellow_22[col].astype('category')

### Numerical Dataset

The float values here is not as big as requiring 64 bits to store them. 64 bits float can store decimal numbers ranging from 2.2E-308 to 1.7E+308 which is definitely way too large for our dataset. Hence, the following features will have ites datatype reduced to lower bit where it will require lesser memory space:
- trip_distance: The elapsed trip distance in miles reported by the taximeter.
- fare_amount: The time-and-distance fare calculated by the meter
- extra: Miscellaneous extras and surcharges which are \\$0.50 and \\$1 rush hour and overnight charges
- tip_amount: This field is automatically populated for credit card tips. 
- total_amount: The total amount charged to passengers. Does not include cash tips.


In [14]:
# To identify the min and max value of the feature and to change the datatype according to the capacity of datatype
for col in ['trip_distance', 'fare_amount', 'extra', 'tip_amount', 'total_amount']:
    c_min = yellow_22[col].min()
    c_max = yellow_22[col].max()
    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
        yellow_22[col] = yellow_22[col].astype(np.float16)
    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
        yellow_22[col] = yellow_22[col].astype(np.float32)
    else:
        yellow_22[col] = yellow_22[col].astype(np.float64)

# To show only 2 decimal places of the float values
pd.options.display.float_format = '{:,.2f}'.format

### Datetime datatype

Both tpep_pickup_datetime & tpep_dropoff_datetime are not correctly categorized. To convert both features to datetime datatype

In [15]:
for col in ['tpep_pickup_datetime', 'tpep_dropoff_datetime']:
    yellow_22[col] = pd.to_datetime(yellow_22[col]) 

### Outcome

In [16]:
# To identify the datatype of each feature after amendment
yellow_22.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
trip_distance                   float32
PULocationID                   category
DOLocationID                   category
fare_amount                     float32
extra                           float16
tip_amount                      float16
total_amount                    float32
dtype: object

In [17]:
# To counter check on the memory usage of the dataset
new_size = yellow_22.memory_usage().sum()/(1024*1024)

print('Original Data Size: {:.2f} MB'.format(ori_size))
print('Optimized Data Size: {:.2f} MB'.format(new_size))
print('Size Reduction: {:.2f}%'.format(100*(ori_size-new_size)/(ori_size)))

Original Data Size: 1795.21 MB
Optimized Data Size: 897.63 MB
Size Reduction: 50.00%


In [18]:
print(yellow_22.shape)
yellow_22.head()

(26144654, 9)


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,extra,tip_amount,total_amount
0,2001-01-01 00:03:14,2001-01-01 01:12:47,20.77,132,230,52.00,0.00,0.00,61.85
1,2001-01-01 00:27:45,2001-01-01 00:34:17,0.88,48,143,6.00,1.00,0.00,10.30
2,2001-01-01 01:02:18,2001-01-01 17:20:19,3.10,237,234,14.00,1.00,0.00,18.30
3,2001-01-01 01:23:51,2001-01-01 01:50:08,4.68,230,231,20.50,1.00,0.00,24.80
4,2001-01-01 01:52:48,2001-01-01 17:37:26,6.95,231,264,22.00,1.00,3.50,29.80


# Data Cleaning

## Only capture data from year 2022
Our project is only targetting for the year 2022

In [19]:
yellow_22 = yellow_22[yellow_22['tpep_pickup_datetime'].dt.year == 2022]
yellow_22 = yellow_22[yellow_22['tpep_dropoff_datetime'].dt.year == 2022]
yellow_22.shape

(26144111, 9)

## To remove rows where fare_amount is 0
This could be due to errors in recording the information. As a result, these rows are removed

In [20]:
yellow_22 = yellow_22[yellow_22['fare_amount'] != 0]

## To remove rows where trip_distant is 0
This could be due to system error and to be removed.

In [21]:
yellow_22 = yellow_22[yellow_22['trip_distance'] != 0]

## To remove unknown Zones
Zones that are not indicated in the map information will be removed which are ZoneID 264 and 265.
Rows where the the `PULocationID` or `DOLocationID` consists of the mentioned zones, it will be removed.

In [22]:
for i in [264,265]:
    yellow_22 = yellow_22[~((yellow_22['PULocationID'] == i) | (yellow_22['DOLocationID'] ==i))]

## To remove rows where pickup_datetime is the same with dropoff_datetime
This could be due to system errors. Hence these rows are removed.


In [23]:
yellow_22 = yellow_22[~(yellow_22['tpep_pickup_datetime'] == yellow_22['tpep_dropoff_datetime'])]


## To remove rows with unrealistic duration and trip distant
To identify these unrealistic datasets which is likely to caused by errors in recording the data, the duration of the ride is deduced to identify the speed of the taxi in sending the passengers. This is to prevent the dataset affecting the demand of the zones at time

### Duration of Ride

To obtain the duration in minute of the taxi ride by deducting the pickup time and dropoff time

In [24]:
yellow_22['duration'] = (yellow_22['tpep_dropoff_datetime'] - yellow_22['tpep_pickup_datetime']).dt.total_seconds()/60

### Speed of Traveling

Dividing the distant of the destination with duration. 

In [25]:
yellow_22['speed_hr'] = yellow_22['trip_distance']/(yellow_22['duration']/60)

The speed limit in NYC is 55 miles per hour, assuming drivers did not follow the speed limit, any speed that is above 80 miles/hr will be removed.

In [26]:
yellow_22 = yellow_22[yellow_22['speed_hr'] <= 80]

In [27]:
yellow_22

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,extra,tip_amount,total_amount,duration,speed_hr
542,2022-01-01 00:00:08,2022-01-01 00:14:14,7.94,138,255,23.00,0.50,4.86,30.41,14.10,33.79
543,2022-01-01 00:00:11,2022-01-01 00:14:29,5.55,132,215,17.50,0.50,0.00,20.05,14.30,23.29
544,2022-01-01 00:00:17,2022-01-01 00:31:04,3.86,170,68,20.00,0.50,4.76,28.56,30.78,7.52
545,2022-01-01 00:00:18,2022-01-01 00:13:27,3.37,79,209,13.00,0.50,3.36,20.16,13.15,15.38
546,2022-01-01 00:00:20,2022-01-01 00:20:51,4.89,68,256,18.00,0.50,6.54,28.34,20.52,14.30
...,...,...,...,...,...,...,...,...,...,...,...
26144648,2022-09-01 00:03:16,2022-09-01 00:09:38,1.72,161,234,7.50,0.50,1.13,12.43,6.37,16.21
26144649,2022-09-01 00:03:18,2022-09-01 00:20:03,4.47,164,7,15.00,0.50,3.00,21.80,16.75,16.01
26144650,2022-09-01 00:04:02,2022-09-01 00:11:30,1.84,48,43,8.00,0.50,2.36,14.16,7.47,14.79
26144651,2022-09-01 00:04:22,2022-09-01 00:28:36,5.31,100,217,20.50,0.50,4.86,29.16,24.23,13.15


## Final Outcome
features that were used to clean the datasets are removed

In [28]:
yellow_22.reset_index(drop = True, inplace = True)
yellow_22.drop(columns = ['duration', 'speed_hr', 
                          'fare_amount', 'extra', 
                          'tip_amount', 'total_amount',
                         'trip_distance'], inplace = True)
print(yellow_22.shape)
yellow_22.head()

(25348245, 4)


,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2022-01-01 00:00:08,2022-01-01 00:14:14,138,255
1,2022-01-01 00:00:11,2022-01-01 00:14:29,132,215
2,2022-01-01 00:00:17,2022-01-01 00:31:04,170,68
3,2022-01-01 00:00:18,2022-01-01 00:13:27,79,209
4,2022-01-01 00:00:20,2022-01-01 00:20:51,68,256


In [29]:
# To counter check on the memory usage of the cleand dataset
new_clean_size = yellow_22.memory_usage().sum()/(1024*1024)

print('Original Data Size: {:.2f} MB'.format(ori_size))
print('Optimized Data Size: {:.2f} MB'.format(new_size))
print('Cleand & Optimized Data Size: {:.2f} MB'.format(new_clean_size))
print('Size Reduction to Optimized Dataset: {:.2f}%'.format(100*(ori_size-new_size)/(ori_size)))
print('Size Reduction to Optimized & Cleaned Dataset: {:.2f}%'.format(100*(ori_size-new_clean_size)/(ori_size)))

Original Data Size: 1795.21 MB
Optimized Data Size: 897.63 MB
Cleand & Optimized Data Size: 483.50 MB
Size Reduction to Optimized Dataset: 50.00%
Size Reduction to Optimized & Cleaned Dataset: 73.07%


# File Export

In [30]:
yellow_22.to_pickle('./datasets/cleaned_yellow_2022.pkl')